# An Adaptive Hybrid Clustering Framework with Iterative Noise Filtering and a Novel DBNS Ratio Validity Measure for Outlier Resilient High-Dimensional Data

In [ ]:
"""
Hybrid Clustering with DBNS (Silhouette-to-Davies-Bouldin Ratio)
================================================================

A comprehensive clustering framework that combines autoencoder-based dimensionality
reduction with a novel hybrid loss function incorporating the DBNS ratio for
improved clustering performance.

Key Features:
- Autoencoder-based 2D latent space representation
- Mahalanobis distance-based outlier detection with adaptive re-weighting
- DBNS-optimized hybrid loss function (Reconstruction + Outlier penalty - DBNS)
- Comprehensive sensitivity analysis for τ, θ, and λ parameters
- Ablation studies to evaluate component contributions
- Extensive visualization suite including t-SNE plots and learning curves
"""

import os
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# ====== Imports ======
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import (
    silhouette_score, davies_bouldin_score, adjusted_rand_score, f1_score,
    roc_curve, auc
)
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import EarlyStopping, Callback

# ====== Configuration ======
# Dataset selection - choose one by uncommenting the appropriate line
file_path = "anemia.csv"     # <--- Update as needed
#file_path = "glass.csv"       # Glass identification dataset
#file_path = "heart.csv"      # Heart disease dataset
#file_path = "hepatitis.csv"  # Hepatitis medical dataset
#file_path = "ionosphere.csv" # Ionosphere radar data
#file_path = "lymphography.csv" # Lymphography medical data
#file_path = "parkinsons.csv" # Parkinson's disease data
#file_path = "pima.csv"       # Pima Indians diabetes dataset
#file_path = "syn1.csv"       # Synthetic dataset 1
#file_path = "syn2.csv"       # Synthetic dataset 2

# Algorithm parameters
enable_outlier_detection = True  # Enable/disable outlier detection module
initial_tau = 1.5                # Initial outlier threshold (Mahalanobis distance)
theta = 0.5                      # Reweighting factor for outlier samples
val_split = 0.20                 # Validation split ratio for autoencoder training
batch_size = 8                   # Batch size for training
max_epochs = 50                  # Maximum training epochs
max_iters = 3                    # Maximum DBNS refinement iterations
base_plots_dir = "Results Hybrid Clustering with DBNS"  # Base directory for results

# ====== Data Loading & Preprocessing ======
dataset_name = os.path.basename(file_path)
dataset_base = os.path.splitext(dataset_name)[0]

# Create dataset-specific directory structure for organized results
plots_dir = os.path.join(base_plots_dir, dataset_base)
os.makedirs(plots_dir, exist_ok=True)

# --- Create organized subfolders for different types of results ---
subdirs = {
    "learning": os.path.join(plots_dir, "learning_curves"),      # Training curves
    "metrics": os.path.join(plots_dir, "cluster_metrics"),       # Clustering metrics
    "tsne": os.path.join(plots_dir, "tsne"),                     # t-SNE visualizations
    "outlier": os.path.join(plots_dir, "outlier"),               # Outlier analysis
    "csv": os.path.join(plots_dir, "csv"),                       # CSV data files
    "pdf": os.path.join(plots_dir, "pdf"),                       # PDF reports
    "sensitivity": os.path.join(plots_dir, "sensitivity"),       # Parameter sensitivity
    "ablation": os.path.join(plots_dir, "ablation")              # Ablation studies
}

# Create all subdirectories
for p in subdirs.values():
    os.makedirs(p, exist_ok=True)

# ====== Runtime timer ======
start_time = time.time()

# Load and preprocess data
print(f"Loading dataset: {dataset_name}")
data = pd.read_csv(file_path)
X = data.iloc[:, :-1].copy()  # Features (all columns except last)
y_true = data.iloc[:, -1].copy()  # Ground truth labels (last column)

# Standardize features for better autoencoder performance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
true_k = len(np.unique(y_true))  # Number of true clusters

print(f"Dataset shape: {X.shape}, Number of clusters: {true_k}")

# ====== Autoencoder Definition ======
input_dim = X_scaled.shape[1]
encoding_dim = 2  # 2D latent space for visualization

# Autoencoder architecture
input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu", name="latent")(input_layer)
decoder = Dense(input_dim, activation="linear")(encoder)

autoencoder = Model(input_layer, decoder, name="autoencoder")
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse", weighted_metrics=["mse"])
encoder_model = Model(inputs=input_layer, outputs=encoder, name="encoder")

print("Autoencoder compiled with 2D latent space")

# ====== Helper Functions ======
def compute_dbns_ratio(X_latent, labels):
    """
    Compute DBNS (Silhouette-to-Davies-Bouldin) ratio.
    
    Parameters:
    -----------
    X_latent : array-like
        Data in latent space
    labels : array-like
        Cluster labels
        
    Returns:
    --------
    float
        DBNS ratio (higher is better)
    """
    if len(np.unique(labels)) < 2:
        return 0.0  # Cannot compute with single cluster
    sil = silhouette_score(X_latent, labels)
    dbi = davies_bouldin_score(X_latent, labels)
    return sil / dbi if dbi > 1e-8 else 0.0

def compute_mahalanobis_scores(X_latent, labels, centroids, eps=1e-6):
    """
    Compute robust Mahalanobis distance to assigned centroid for each point.
    
    Parameters:
    -----------
    X_latent : array-like
        Data in latent space
    labels : array-like
        Cluster assignments
    centroids : array-like
        Cluster centroids
    eps : float
        Small value for numerical stability
        
    Returns:
    --------
    array
        Mahalanobis distances for each sample
    """
    n, d = X_latent.shape
    scores = np.zeros(n, dtype=float)
    for c in np.unique(labels):
        idxs = np.where(labels == c)[0]
        Xc = X_latent[idxs]
        mu = centroids[c].ravel()
        if Xc.shape[0] < 2:
            inv_cov = np.linalg.pinv(np.eye(d) * eps)  # Identity matrix for small clusters
        else:
            cov = np.cov(Xc, rowvar=False)
            cov = np.nan_to_num(cov) + eps * np.eye(cov.shape[0])  # Regularize covariance
            inv_cov = np.linalg.pinv(cov)
        diffs = Xc - mu
        scores[idxs] = np.sqrt(np.einsum('ni,ij,nj->n', diffs, inv_cov, diffs))
    return scores

def adaptive_reweight(scores, tau, theta=0.5):
    """
    Apply adaptive reweighting based on outlier scores.
    
    Parameters:
    -----------
    scores : array-like
        Outlier scores (Mahalanobis distances)
    tau : float
        Threshold for outlier detection
    theta : float
        Weight for outlier samples
        
    Returns:
    --------
    array
        Sample weights
    """
    w = np.ones(len(scores))
    w[scores > tau] = theta  # Reduce weight for outliers
    return w

def dynamic_tau(scores, lambda_coeff=1.0):
    """
    Compute dynamic threshold τ = μ + λσ for outlier detection.
    
    Parameters:
    -----------
    scores : array-like
        Outlier scores
    lambda_coeff : float
        Multiplier for standard deviation
        
    Returns:
    --------
    float
        Dynamic threshold value
    """
    mu = float(np.mean(scores))
    sd = float(np.std(scores, ddof=1)) if len(scores) > 1 else 0.0
    return mu + lambda_coeff * sd

def hubness_score(X, k=5):
    """
    Compute hubness score (standard deviation of neighbor counts).
    
    Parameters:
    -----------
    X : array-like
        Input data
    k : int
        Number of neighbors
        
    Returns:
    --------
    float
        Hubness score
    """
    nbrs = NearestNeighbors(n_neighbors=k).fit(X)
    idx = nbrs.kneighbors(X, return_distance=False)
    hubs = np.bincount(idx.flatten(), minlength=len(X))
    return float(np.std(hubs))

def _auto_perplexity(n):
    """
    Automatically determine perplexity for t-SNE based on dataset size.
    
    Parameters:
    -----------
    n : int
        Number of samples
        
    Returns:
    --------
    int
        Perplexity value
    """
    return int(max(5, min(30, n // 3 - 1))) if n > 20 else 5

def plot_tsne_clusters_save(X_latent, labels_km, title_prefix, dataset_base, pdf, random_state=42):
    """
    Create and save t-SNE visualization colored by KMeans clusters.
    
    Parameters:
    -----------
    X_latent : array-like
        Data in latent space
    labels_km : array-like
        KMeans cluster labels
    title_prefix : str
        Plot title prefix
    dataset_base : str
        Dataset name for file naming
    pdf : PdfPages object
        PDF object for saving
    random_state : int
        Random seed for reproducibility
    """
    n = X_latent.shape[0]
    perplexity = _auto_perplexity(n)
    tsne = TSNE(n_components=2, init="pca", learning_rate="auto",
                perplexity=perplexity, random_state=random_state)
    Z = tsne.fit_transform(X_latent)

    plt.figure()
    plt.scatter(Z[:, 0], Z[:, 1], c=labels_km, s=16, alpha=0.9, cmap="tab10")
    plt.title(f"{title_prefix} – t-SNE (KMeans labels)\nperplexity={perplexity}")
    plt.xticks([]); plt.yticks([]); plt.tight_layout()

    base = os.path.join(subdirs["tsne"], f"{dataset_base}_{title_prefix.replace(' ','_').lower()}_tsne_kmeans")
    plt.savefig(base + ".jpeg", dpi=200)
    plt.savefig(base + ".pdf")
    pdf.savefig()
    plt.close()

# ====== Sensitivity Analysis Functions ======
def run_with_theta(X_scaled, y_true, tau, theta, epochs=30):
    """
    Run a complete clustering iteration with specific tau and theta parameters.
    
    Parameters:
    -----------
    X_scaled : array-like
        Standardized input features
    y_true : array-like
        Ground truth labels
    tau : float
        Outlier threshold
    theta : float
        Reweighting factor
    epochs : int
        Training epochs
        
    Returns:
    --------
    tuple
        (DBNS, ARI, F1, labels) metrics
    """
    input_dim = X_scaled.shape[1]
    encoding_dim = 2
    
    # Create fresh autoencoder for each run
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(encoding_dim, activation="relu", name="latent")(input_layer)
    decoder = Dense(input_dim, activation="linear")(encoder)
    autoencoder = Model(input_layer, decoder, name="autoencoder")
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse", weighted_metrics=["mse"])
    encoder_model = Model(inputs=input_layer, outputs=encoder, name="encoder")
    
    # Train autoencoder
    autoencoder.fit(X_scaled, X_scaled, epochs=epochs, batch_size=8, verbose=0)
    
    # Get latent representation and cluster
    X_latent = encoder_model.predict(X_scaled, verbose=0)
    kmeans = KMeans(n_clusters=len(np.unique(y_true)), random_state=42)
    labels = kmeans.fit_predict(X_latent)
    centroids = kmeans.cluster_centers_
    
    # Compute outlier scores and apply weighting
    scores = compute_mahalanobis_scores(X_latent, labels, centroids)
    weights = adaptive_reweight(scores, tau, theta=theta)
    
    # Retrain with sample weights
    autoencoder.fit(X_scaled, X_scaled, epochs=10, batch_size=8, 
                   sample_weight=weights, verbose=0)
    
    # Final evaluation
    X_latent_final = encoder_model.predict(X_scaled, verbose=0)
    kmeans_final = KMeans(n_clusters=len(np.unique(y_true)), random_state=42)
    labels_final = kmeans_final.fit_predict(X_latent_final)
    
    # Compute final metrics
    if len(np.unique(labels_final)) > 1:
        dbns = compute_dbns_ratio(X_latent_final, labels_final)
    else:
        dbns = 0.0
        
    ari = adjusted_rand_score(y_true, labels_final)
    f1m = f1_score(y_true, labels_final, average="macro")
    
    return dbns, ari, f1m, labels_final

def plot_lines_with_dataset_name_in_filename(df, x, metrics, title, xlabel, ylabel,
                                             pdf_filename_base, dataset_name, output_dir):
    """
    Create and save line plots for sensitivity analysis.
    
    Parameters:
    -----------
    df : DataFrame
        Data to plot
    x : str
        x-axis column name
    metrics : list
        y-axis metrics to plot
    title : str
        Plot title
    xlabel : str
        x-axis label
    ylabel : str
        y-axis label
    pdf_filename_base : str
        Base filename for PDF
    dataset_name : str
        Dataset name for file naming
    output_dir : str
        Output directory
    """
    plt.figure(figsize=(6, 4))
    line_styles = ['-', '--', ':']
    for i, m in enumerate(metrics):
        plt.plot(df[x], df[m], linestyle=line_styles[i % len(line_styles)],
                 marker='o', label=m)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.tight_layout()

    pdf_filename = os.path.join(
        output_dir, f"{pdf_filename_base.split('.pdf')[0]}_{dataset_name}.pdf"
    )
    plt.savefig(pdf_filename)
    plt.close()

def sensitivity_dashboard(csv_file,
                          tau_grid=[1.2, 1.3, 1.4, 1.5, 1.6],
                          theta_grid=[0.0, 0.25, 0.5, 0.75, 1.0],
                          epochs=30):
    """
    Run comprehensive sensitivity analysis for τ and θ parameters.
    
    Parameters:
    -----------
    csv_file : str
        Path to dataset file
    tau_grid : list
        Values of τ to test
    theta_grid : list
        Values of θ to test
    epochs : int
        Training epochs per configuration
    """
    dataset_name = os.path.splitext(os.path.basename(csv_file))[0]
    data = pd.read_csv(csv_file)
    X = data.iloc[:, :-1].values
    y_true = data.iloc[:, -1].values
    X_scaled = StandardScaler().fit_transform(X)

    out_dir = subdirs["sensitivity"]

    # τ sweep ---------------------------------------------------------------
    print("Running τ sensitivity analysis...")
    tau_results = []
    for tau in tau_grid:
        dbns, ari, f1m, _ = run_with_theta(X_scaled, y_true, tau, theta=1.0, epochs=epochs)
        tau_results.append({"tau": tau, "DBNS": dbns, "ARI": ari, "F1": f1m})
    tau_df = pd.DataFrame(tau_results)
    tau_df.to_csv(os.path.join(out_dir, f"{dataset_name}_tau_sensitivity.csv"), index=False)

    # θ sweep ---------------------------------------------------------------
    print("Running θ sensitivity analysis...")
    theta_results = []
    for th in theta_grid:
        dbns, ari, f1m, _ = run_with_theta(X_scaled, y_true, tau=1.5, theta=th, epochs=epochs)
        theta_results.append({"theta": th, "DBNS": dbns, "ARI": ari, "F1": f1m})
    theta_df = pd.DataFrame(theta_results)
    theta_df.to_csv(os.path.join(out_dir, f"{dataset_name}_theta_sensitivity.csv"), index=False)

    # PLOTS -----------------------------------------------------------------
    plot_lines_with_dataset_name_in_filename(
        tau_df, "tau", ["DBNS", "ARI", "F1"],
        "Performance vs τ", "τ", "Metric value",
        "performance_vs_tau.pdf", dataset_name, out_dir
    )
    plot_lines_with_dataset_name_in_filename(
        theta_df, "theta", ["DBNS", "ARI", "F1"],
        "Performance vs θ", "θ", "Metric value",
        "performance_vs_theta.pdf", dataset_name, out_dir
    )
    print(f"Sensitivity analysis complete for {dataset_name}. Results in {out_dir}")

# ====== λ-SENSITIVITY SWEEP (tau = μ + λσ) ===============================================
def lambda_sweep(csv_file,
                 lambdas=(0.5, 0.8, 1.0, 1.2, 1.5),
                 n_runs=5,
                 theta=0.5,
                 epochs=30,
                 fine_tune_epochs=10,
                 choose_by="DBNS"):
    """
    Perform λ-sensitivity analysis for dynamic thresholding.
    
    For each λ, train an AE, cluster in latent, compute Mahalanobis scores,
    set τ = μ + λσ, re-weight (θ below/above τ), fine-tune, re-cluster and record:
      - DBNS (Silhouette/DBI)
      - ARI
      - Outlier% (= fraction scores>τ * 100)
    
    Parameters:
    -----------
    csv_file : str
        Path to dataset file
    lambdas : tuple
        λ values to test
    n_runs : int
        Number of runs per λ for statistical significance
    theta : float
        Reweighting factor
    epochs : int
        Initial training epochs
    fine_tune_epochs : int
        Fine-tuning epochs with weights
    choose_by : str
        Metric to optimize ("DBNS" or "ARI")
        
    Prints:
    -------
    [λ-sweep] Suggested λ = ... (S/DB=...±..., ARI=...±..., Outlier=...%±...%)
    """
    ds = os.path.splitext(os.path.basename(csv_file))[-2]
    data = pd.read_csv(csv_file)
    X = data.iloc[:, :-1].values
    y_true = data.iloc[:, -1].values
    X_scaled = StandardScaler().fit_transform(X)

    out_root = os.path.join(subdirs["sensitivity"], "lambda_sweep", ds)
    os.makedirs(out_root, exist_ok=True)

    rng = np.random.RandomState(42)

    def _one_run(lambda_coeff):
        """Run a single iteration with specific lambda coefficient."""
        # Fresh AE per run
        inp = Input(shape=(X_scaled.shape[1],))
        z   = Dense(2, activation="relu")(inp)
        out = Dense(X_scaled.shape[1], activation="linear")(z)
        ae  = Model(inp, out)
        ae.compile(optimizer=Adam(1e-3), loss="mse")
        enc = Model(inp, z)

        ae.fit(X_scaled, X_scaled, epochs=epochs, batch_size=8,
               validation_split=0.2, shuffle=True, verbose=0)

        Z = enc.predict(X_scaled, verbose=0)
        km = KMeans(n_clusters=len(np.unique(y_true)), random_state=rng.randint(0, 10_000)).fit(Z)
        labels = km.labels_
        centers = km.cluster_centers_

        scores = compute_mahalanobis_scores(Z, labels, centers)
        tau_l = dynamic_tau(scores, lambda_coeff=lambda_coeff)
        w = adaptive_reweight(scores, tau_l, theta=theta)

        # brief fine-tune with weights
        if fine_tune_epochs > 0:
            ae.fit(X_scaled, X_scaled, sample_weight=w, epochs=fine_tune_epochs,
                   batch_size=8, validation_split=0.2, shuffle=True, verbose=0)

        # final metrics
        Z2 = enc.predict(X_scaled, verbose=0)
        km2 = KMeans(n_clusters=len(np.unique(y_true)), random_state=rng.randint(0, 10_000)).fit(Z2)
        lab2 = km2.labels_

        if len(np.unique(lab2)) > 1:
            dbns = compute_dbns_ratio(Z2, lab2)
        else:
            dbns = 0.0
        ari = adjusted_rand_score(y_true, lab2)
        outlier_rate = float(np.mean(scores > tau_l) * 100.0)  # %
        return dbns, ari, outlier_rate

    # run sweep
    print(f"Running λ-sweep with {n_runs} runs per λ value...")
    rows = []
    for lam in lambdas:
        dbns_runs = []
        ari_runs = []
        out_runs = []
        for _ in range(n_runs):
            d, a, o = _one_run(lam)
            dbns_runs.append(d); ari_runs.append(a); out_runs.append(o)
        rows.append({
            "lambda": lam,
            "DBNS_mean": np.mean(dbns_runs), "DBNS_std": np.std(dbns_runs, ddof=0),
            "ARI_mean":  np.mean(ari_runs),  "ARI_std":  np.std(ari_runs,  ddof=0),
            "Outlier_mean": np.mean(out_runs), "Outlier_std": np.std(out_runs, ddof=0)
        })

    res_df = pd.DataFrame(rows)
    csv_path = os.path.join(out_root, f"{ds}_lambda_sweep.csv")
    res_df.to_csv(csv_path, index=False)

    # pick best λ based on specified metric
    if choose_by.upper() == "ARI":
        best_idx = res_df["ARI_mean"].values.argmax()
    else:
        best_idx = res_df["DBNS_mean"].values.argmax()
    best = res_df.iloc[best_idx]

    # create visualization plots
    plt.figure(figsize=(6,4))
    plt.errorbar(res_df["lambda"], res_df["DBNS_mean"], yerr=res_df["DBNS_std"], marker='o', label="DBNS")
    plt.xlabel("λ"); plt.ylabel("DBNS"); plt.title(f"DBNS vs λ — {ds}")
    plt.tight_layout(); plt.savefig(os.path.join(out_root, f"{ds}_lambda_vs_dbns.pdf")); plt.close()

    plt.figure(figsize=(6,4))
    plt.errorbar(res_df["lambda"], res_df["ARI_mean"], yerr=res_df["ARI_std"], marker='o', label="ARI")
    plt.xlabel("λ"); plt.ylabel("ARI"); plt.title(f"ARI vs λ — {ds}")
    plt.tight_layout(); plt.savefig(os.path.join(out_root, f"{ds}_lambda_vs_ari.pdf")); plt.close()

    plt.figure(figsize=(6,4))
    plt.errorbar(res_df["lambda"], res_df["Outlier_mean"], yerr=res_df["Outlier_std"], marker='o', label="Outlier %")
    plt.xlabel("λ"); plt.ylabel("Outlier (%)"); plt.title(f"Outlier% vs λ — {ds}")
    plt.tight_layout(); plt.savefig(os.path.join(out_root, f"{ds}_lambda_vs_outlier.pdf")); plt.close()

    # print summary line
    print(f"[λ-sweep] Suggested λ = {best['lambda']:.2f} "
          f"(S/DB={best['DBNS_mean']:.3f}±{best['DBNS_std']:.3f}, "
          f"ARI={best['ARI_mean']:.3f}±{best['ARI_std']:.3f}, "
          f"Outlier={best['Outlier_mean']:.1f}%±{best['Outlier_std']:.1f}%)")
    print(f"Saved λ-sweep CSV/plots in: {out_root}")

# ====== Ablation Study Functions ======
def run_clustering_variant(X_scaled, y_true, tau, theta, remove_dbns=False, 
                          remove_reweighting=False, epochs=50):
    """
    Run clustering with specific ablation conditions.
    
    Parameters:
    -----------
    X_scaled : array-like
        Standardized features
    y_true : array-like
        Ground truth labels
    tau : float
        Outlier threshold
    theta : float
        Reweighting factor
    remove_dbns : bool
        Whether to remove DBNS from loss
    remove_reweighting : bool
        Whether to remove reweighting
    epochs : int
        Training epochs
        
    Returns:
    --------
    dict
        Performance metrics for this variant
    """
    input_dim = X_scaled.shape[1]
    encoding_dim = 2
    
    # Create new autoencoder
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(encoding_dim, activation="relu", name="latent")(input_layer)
    decoder = Dense(input_dim, activation="linear")(encoder)
    autoencoder = Model(input_layer, decoder, name="autoencoder")
    autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss="mse", weighted_metrics=["mse"])
    encoder_model = Model(inputs=input_layer, outputs=encoder, name="encoder")
    
    # Train autoencoder
    autoencoder.fit(X_scaled, X_scaled, epochs=epochs, batch_size=8, verbose=0)
    
    # Get latent representation
    X_latent = encoder_model.predict(X_scaled, verbose=0)
    kmeans = KMeans(n_clusters=len(np.unique(y_true)), random_state=42)
    labels = kmeans.fit_predict(X_latent)
    centroids = kmeans.cluster_centers_
    
    # Compute scores
    scores = compute_mahalanobis_scores(X_latent, labels, centroids)
    
    # Apply ablation conditions
    if remove_reweighting:
        weights = np.ones(len(scores))  # No reweighting
    else:
        weights = adaptive_reweight(scores, tau, theta=theta)
    
    # Retrain with weights
    autoencoder.fit(X_scaled, X_scaled, epochs=10, batch_size=8, 
                   sample_weight=weights, verbose=0)
    
    # Final evaluation
    X_latent_final = encoder_model.predict(X_scaled, verbose=0)
    kmeans_final = KMeans(n_clusters=len(np.unique(y_true)), random_state=42)
    labels_final = kmeans_final.fit_predict(X_latent_final)
    
    # Compute metrics
    if len(np.unique(labels_final)) > 1:
        sil = silhouette_score(X_latent_final, labels_final)
        dbi = davies_bouldin_score(X_latent_final, labels_final)
        dbns = sil / dbi if dbi > 1e-8 else 0.0
    else:
        sil = 0.0
        dbi = 0.0
        dbns = 0.0
        
    ari = adjusted_rand_score(y_true, labels_final)
    f1m = f1_score(y_true, labels_final, average="macro")
    
    # If DBNS is removed, we don't use it in any loss (still reported as a metric)
    hybrid_loss = float(np.mean(scores**2)) - (0.0 if remove_dbns else dbns)
    
    return {
        'DBNS': dbns,
        'ARI': ari,
        'F1': f1m,
        'Silhouette': sil,
        'DBI': dbi,
        'HybridLoss': hybrid_loss
    }

def ablation_experiment(csv_file, tau=1.5, theta=0.5, epochs=50):
    """
    Run comprehensive ablation study comparing method variants.
    
    Parameters:
    -----------
    csv_file : str
        Path to dataset file
    tau : float
        Outlier threshold
    theta : float
        Reweighting factor
    epochs : int
        Training epochs
        
    Returns:
    --------
    DataFrame
        Ablation study results
    """
    dataset_name = os.path.splitext(os.path.basename(csv_file))[0]
    data = pd.read_csv(csv_file)
    X = data.iloc[:, :-1].values
    y_true = data.iloc[:, -1].values
    X_scaled = StandardScaler().fit_transform(X)

    out_dir = subdirs["ablation"]
    
    # 1. Full method (with DBNS and re-weighting)
    print("Running Full method variant...")
    res_full = run_clustering_variant(X_scaled, y_true, tau, theta,
                                      remove_dbns=False, remove_reweighting=False, epochs=epochs)
    # 2. Remove DBNS (only S/DB provided, DBNS not used)
    print("Running No-DBNS variant...")
    res_no_dbns = run_clustering_variant(X_scaled, y_true, tau, theta,
                                         remove_dbns=True, remove_reweighting=False, epochs=epochs)
    # 3. Remove re-weighting (force theta=1.0)
    print("Running No-Reweighting variant...")
    res_no_reweight = run_clustering_variant(X_scaled, y_true, tau, 1.0,
                                             remove_dbns=False, remove_reweighting=True, epochs=epochs)
    
    results = pd.DataFrame([
        {'Variant': 'Full', **res_full},
        {'Variant': 'No DBNS', **res_no_dbns},
        {'Variant': 'No Re-weighting', **res_no_reweight}
    ])

    # Save results
    results.to_csv(os.path.join(out_dir, f"{dataset_name}_ablation_results.csv"), index=False)
    
    # Create ablation plot
    plt.figure(figsize=(10, 6))
    metrics = ['DBNS', 'ARI', 'F1', 'Silhouette']
    x_pos = np.arange(len(metrics))
    width = 0.25
    
    full_vals = [results[results['Variant'] == 'Full'][m].values[0] for m in metrics]
    no_dbns_vals = [results[results['Variant'] == 'No DBNS'][m].values[0] for m in metrics]
    no_reweight_vals = [results[results['Variant'] == 'No Re-weighting'][m].values[0] for m in metrics]
    
    plt.bar(x_pos - width, full_vals, width, label='Full Method')
    plt.bar(x_pos, no_dbns_vals, width, label='No DBNS')
    plt.bar(x_pos + width, no_reweight_vals, width, label='No Re-weighting')
    
    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title(f'Ablation Study - {dataset_name}')
    plt.xticks(x_pos, metrics)
    plt.legend()
    plt.tight_layout()
    
    # Save plot
    plt.savefig(os.path.join(out_dir, f"{dataset_name}_ablation_study.pdf"))
    plt.savefig(os.path.join(out_dir, f"{dataset_name}_ablation_study.jpeg"), dpi=200)
    plt.close()
    
    print(f"Ablation study complete for {dataset_name}")
    print(results)
    
    return results

# ====== Callback to log clustering metrics per epoch ======
class ClusteringMetricsCallback(Callback):
    """
    Custom callback to track clustering metrics during autoencoder training.
    """
    def __init__(self, X_scaled, y_true, encoder_model, true_k, save_prefix):
        super().__init__()
        self.X = X_scaled
        self.y_true = np.asarray(y_true)
        self.encoder_model = encoder_model
        self.true_k = true_k
        self.save_prefix = save_prefix
        self.history = {"epoch": [], "ARI": [], "F1": [], "DBNS": []}

    def on_epoch_end(self, epoch, logs=None):
        """Compute clustering metrics at the end of each epoch."""
        X_latent = self.encoder_model.predict(self.X, verbose=0)
        kmeans = KMeans(n_clusters=self.true_k, random_state=42)
        labels = kmeans.fit_predict(X_latent)
        self.history["epoch"].append(epoch + 1)
        self.history["ARI"].append(adjusted_rand_score(self.y_true, labels))
        self.history["F1"].append(f1_score(self.y_true, labels, average="macro"))
        self.history["DBNS"].append(compute_dbns_ratio(X_latent, labels))

    def on_train_end(self, logs=None):
        """Save clustering metrics history at the end of training."""
        hist_df = pd.DataFrame(self.history)
        csv_path = os.path.join(subdirs["csv"], self.save_prefix + "_cluster_metrics_per_epoch.csv")
        hist_df.to_csv(csv_path, index=False)

# ====== Main Training Pipeline ======
print("\n" + "="*60)
print("STARTING HYBRID CLUSTERING WITH DBNS")
print("="*60)

# Initialize callback and early stopping
cm_prefix = dataset_base
metrics_cb = ClusteringMetricsCallback(X_scaled, y_true, encoder_model, true_k, cm_prefix)
early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

# Train autoencoder
print("Training autoencoder...")
history = autoencoder.fit(
    X_scaled, X_scaled,
    epochs=max_epochs, batch_size=batch_size,
    validation_split=val_split, shuffle=True, verbose=0,
    callbacks=[early_stop, metrics_cb]
)

# ====== Save training history CSV ======
hist_df = pd.DataFrame(history.history)
hist_df.to_csv(os.path.join(subdirs["csv"], f"{dataset_base}_ae_history.csv"), index=False)

# ====== Combined PDF path ======
pdf_path = os.path.join(subdirs["pdf"], f"{dataset_base}_results.pdf")
pdf = PdfPages(pdf_path)

# ====== Learning curves (learning/ + combined PDF) ======
plt.figure()
plt.plot(history.history["loss"], label="Train Loss (MSE)")
plt.plot(history.history["val_loss"], label="Val Loss (MSE)")
plt.xlabel("Epoch"); plt.ylabel("Loss")
plt.title(f"Learning Curves – {dataset_base}")
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(subdirs["learning"], f"{dataset_base}_learning_curves.jpeg"), dpi=200)
plt.savefig(os.path.join(subdirs["learning"], f"{dataset_base}_learning_curves.pdf"))
pdf.savefig(); plt.close()

# ====== Clustering metrics over epochs (metrics/ + combined PDF) ======
cm_df = pd.read_csv(os.path.join(subdirs["csv"], cm_prefix + "_cluster_metrics_per_epoch.csv"))
plt.figure()
plt.plot(cm_df["epoch"], cm_df["ARI"], label="ARI")
plt.plot(cm_df["epoch"], cm_df["F1"], label="F1 (macro)")
plt.plot(cm_df["epoch"], cm_df["DBNS"], label="DBNS")
plt.xlabel("Epoch"); plt.ylabel("Score")
plt.title(f"Clustering Metrics – {dataset_base}")
plt.legend(); plt.tight_layout()
plt.savefig(os.path.join(subdirs["metrics"], f"{dataset_base}_cluster_metrics.jpeg"), dpi=200)
plt.savefig(os.path.join(subdirs["metrics"], f"{dataset_base}_cluster_metrics.pdf"))
pdf.savefig(); plt.close()

# ====== Initial latent, clustering, and scores ======
print("Computing initial clustering metrics...")
X_latent = encoder_model.predict(X_scaled, verbose=0)
kmeans = KMeans(n_clusters=true_k, random_state=42)
labels = kmeans.fit_predict(X_latent)
centroids = kmeans.cluster_centers_
scores = compute_mahalanobis_scores(X_latent, labels, centroids)
tau = initial_tau

# ====== t-SNE BEFORE (tsne/ + combined PDF) ======
plot_tsne_clusters_save(X_latent, labels, "Before DBNS", dataset_base, pdf)

# ====== Baseline metrics BEFORE refinement ======
if len(np.unique(labels)) > 1:
    silhouette = silhouette_score(X_latent, labels)
    dbi = davies_bouldin_score(X_latent, labels)
    s_db_ratio = silhouette / dbi if dbi != 0 else 0.0
else:
    silhouette = 0.0; dbi = 0.0; s_db_ratio = 0.0

ari = adjusted_rand_score(y_true, labels)
f1 = f1_score(y_true, labels, average='macro')
hub_score = hubness_score(X_latent)

roc_auc = None
if enable_outlier_detection and len(np.unique(y_true)) == 2:
    fpr, tpr, _ = roc_curve(y_true, scores)
    roc_auc = auc(fpr, tpr)

# ====== Iterative DBNS refinement ======
refine_metrics = []
for refine_iter in range(max_iters):
    tau = dynamic_tau(scores, lambda_coeff=1.0)
    weights = adaptive_reweight(scores, tau, theta=theta)

    # (Optional) brief re-train with weights:
    # autoencoder.fit(X_scaled, X_scaled, epochs=10, batch_size=batch_size, sample_weight=weights, verbose=0)

    X_latent = encoder_model.predict(X_scaled, verbose=0)
    kmeans = KMeans(n_clusters=true_k, random_state=42)
    labels = kmeans.fit_predict(X_latent)
    centroids = kmeans.cluster_centers_
    scores = compute_mahalanobis_scores(X_latent, labels, centroids)

    if len(np.unique(labels)) > 1:
        silscore = silhouette_score(X_latent, labels)
        dbi = davies_bouldin_score(X_latent, labels)
        dbns = silscore / dbi if dbi > 1e-8 else 0.0
    else:
        dbns = 0.0

    ari_it = adjusted_rand_score(y_true, labels)
    f1m_it = f1_score(y_true, labels, average="macro")

    dists = np.linalg.norm(X_latent - centroids[labels], axis=1)
    recons_loss = np.average(dists**2, weights=weights)
    outlier_penalty = float(np.sum(scores[weights < 1.0]))
    beta = 1.0; alpha = 0.1
    hybrid_loss = recons_loss + alpha * outlier_penalty - beta * dbns

    refine_metrics.append({
        "iter": refine_iter + 1,
        "tau": float(tau),
        "DBNS": float(dbns),
        "ARI": float(ari_it),
        "F1": float(f1m_it),
        "HybridLoss": float(hybrid_loss)
    })
    print(f"[Refine {refine_iter+1}] tau={tau:.3f}, DBNS={dbns:.3f}, ARI={ari_it:.3f}, F1={f1m_it:.3f}, HybridLoss={hybrid_loss:.3f}")

# Save refinement metrics CSV
pd.DataFrame(refine_metrics).to_csv(os.path.join(subdirs["csv"], f"{dataset_base}_refine_metrics.csv"), index=False)

# ====== Final t-SNE AFTER (tsne/ + combined PDF) ======
X_latent_final = encoder_model.predict(X_scaled, verbose=0)
kmeans_final = KMeans(n_clusters=true_k, random_state=42)
labels_final = kmeans_final.fit_predict(X_latent_final)
plot_tsne_clusters_save(X_latent_final, labels_final, "After DBNS", dataset_base, pdf)

# ====== Outlier Score Distribution (outlier/ + combined PDF) ======
plt.figure()
plt.hist(scores, bins=40)
plt.xlabel("Mahalanobis distance (anomaly score)")
plt.ylabel("Count")
plt.title(f"Outlier Score Histogram – {dataset_base}")
plt.tight_layout()
base = os.path.join(subdirs["outlier"], f"{dataset_base}_outlier_scores_hist")
plt.savefig(base + ".jpeg", dpi=200); plt.savefig(base + ".pdf"); pdf.savefig(); plt.close()

score_by_cluster = [scores[labels_final == c] for c in np.unique(labels_final)]
plt.figure()
plt.boxplot(score_by_cluster, showfliers=True)
plt.xlabel("Cluster index"); plt.ylabel("Mahalanobis distance (anomaly score)")
plt.title(f"Outlier Scores by Cluster – {dataset_base}")
plt.tight_layout()
base = os.path.join(subdirs["outlier"], f"{dataset_base}_outlier_scores_boxplot")
plt.savefig(base + ".jpeg", dpi=200); plt.savefig(base + ".pdf"); pdf.savefig(); plt.close()

# ====== Run Sensitivity Analyses ======
sensitivity_dashboard(file_path)                       # existing τ/θ sweeps
lambda_sweep(file_path, lambdas=(0.5,0.8,1.0,1.2,1.5), # NEW λ-sweep
             n_runs=5, theta=theta, epochs=30, fine_tune_epochs=10, choose_by="DBNS")

# ====== Run Ablation Study ======
ablation_experiment(file_path, tau=initial_tau, theta=theta, epochs=30)

# ====== Runtime log & console summary ======
end_time = time.time()
elapsed_time = round(end_time - start_time, 2)
timestamp = time.strftime("[%Y-%m-%d %H:%M:%S]")
with open(os.path.join(subdirs["csv"], "execution_log.txt"), "a", encoding="utf-8") as f:
    f.write(f"{timestamp} Executed clustering on {dataset_name} in {elapsed_time} seconds.\n")

# ====== Display final metrics ======
print(f"\nDataset: {dataset_name}")
print(f"Silhouette: {silhouette:.3f}  DBI: {dbi:.3f}  DBNS: {s_db_ratio:.3f}")
print(f"ARI: {ari:.3f}  F1: {f1:.3f}  Hubness: {hub_score:.3f}")
if enable_outlier_detection and roc_auc is not None:
    print(f"AUC (Outlier): {roc_auc:.3f}")
elif enable_outlier_detection:
    print("AUC (Outlier): skipped; requires binary ground truth")

# ====== Close combined PDF ======
pdf.close()
print(f"\nAll results saved in: {plots_dir}")
print(f"Combined PDF report: {pdf_path}")


Loading dataset: anemia.csv
Dataset shape: (1421, 5), Number of clusters: 2

Autoencoder compiled with 2D latent space

STARTING HYBRID CLUSTERING WITH DBNS
Training autoencoder...


Computing initial clustering metrics...
[Refine 1] tau=1.895, DBNS=0.523, ARI=0.060, F1=0.371, HybridLoss=42.953
[Refine 2] tau=1.895, DBNS=0.523, ARI=0.060, F1=0.371, HybridLoss=42.953
[Refine 3] tau=1.895, DBNS=0.523, ARI=0.060, F1=0.371, HybridLoss=42.953
Running τ sensitivity analysis...
Running θ sensitivity analysis...
